<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Question-5-1-Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow_datasets

In [ ]:
import io
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd 
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k', split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                          with_info=True, as_supervised=False)


In [ ]:
df = pd.read_csv("pubmed_results.csv")
df.dropna(inplace=True)
df

,year,month,title,day,abstract
1,2012,7,Golimumab for the treatment of psoriatic arthr...,5,This paper presents a summary of the evidence ...
2,2012,7,Liraglutide for the treatment of type 2 diabetes.,5,This paper presents a summary of the evidence ...
3,2012,7,Trabectedin for the treatment of relapsed ovar...,5,The paper presents a summary of the evidence r...
4,2012,7,Ofatumumab for the treatment of chronic lympho...,5,This paper presents a summary of the evidence ...
5,2012,7,Denosumab for the prevention of osteoporotic f...,5,This paper presents a summary of the evidence ...
...,...,...,...,...,...
995,2020,12,Using Whatsapp for the Consultation of Covid-1...,4,"Telephone, internet-connected devices (phablet..."
996,2020,12,A comprehensive study on ozone pollution in a ...,4,Tropospheric ozone (O3) pollution has b
997,2020,12,Long-term exposure to low-level air pollution ...,4,Air pollution has been suggested as a risk fac...
998,2020,12,Radiation-induced lens opacities: Epidemiologi...,4,"In 2011, the International Commission on Radio..."


In [ ]:
MAX_SEQUENCE_LENGTH = 50
VOCAB_SIZE = 10000
int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

def int_vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text)

In [ ]:
int_vectorize_layer.adapt(df['title'].to_numpy())


In [ ]:
int_vectorize_layer.get_vocabulary()

In [ ]:
int_train_ds = df['title'].map(int_vectorize_text)

In [ ]:
all_sentences = df['title'].to_numpy()
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

In [ ]:
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [ ]:
all_words

In [ ]:
model = Word2Vec(all_words, min_count=2, size=50)

v1 = model.wv['mediate']
model.similarity('China', 'ozone')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.47380164

In [ ]:
model.wv.vocab['Intervention']


In [ ]:
out_vectors = io.open('vecs.tsv', 'w', encoding='utf-8')
out_metadata = io.open('meta.tsv', 'w', encoding='utf-8')

my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    out_metadata.write(key + '\n')
    out_vectors.write('\t'.join([str(x) for x in model.wv.get_vector(key)]) + '\n')
out_metadata.close()
out_vectors.close()
    # my_dict[key] = model.wv.get_vector(key)
    # # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)